# Q1 Training Datasets

Training Datasets (TDS) in EOTDL are categorized into different [quality levels](https://eotdl.com/docs/datasets/quality), which in turn will impact the range of functionality that will be available for each dataset.

In this tutorial you will learn about Q1 datsets, datasets with STAC metadata. 

To ingest a Q1 datasets you will need its STAC metadata.

Some datasets already have STAC metadata, and can be ingested directly into EOTDL. However, in case that your dataset does not have STAC metadata but you want to ingest it as a Q1 dataset, the EOTDL library also offers functionality to create the metadata. Let's see an example using the EuroSAT dataset. 

In [18]:
import eotdl 

eotdl.__version__

'2024.04.25'

In [1]:
from eotdl.datasets import download_dataset

download_dataset("EuroSAT", version=1, path="data", force=True)

100%|██████████| 1.93G/1.93G [00:29<00:00, 69.8MiB/s]
100%|██████████| 1/1 [00:29<00:00, 29.96s/file]


'data/EuroSAT/v1'

In [35]:
!ls data/EuroSAT/v1

EuroSAT.zip  README.md


In [36]:
!unzip -q data/EuroSAT/v1/EuroSAT.zip -d data/EuroSAT

The EuroSAT dataset contains satellite images for classification, i.e. each image has one label associated. In this case, the label can be extracted from the folder structure.

In [37]:
import os 

base_path = 'data/EuroSAT'
images_path = base_path + '/ds/images/remote_sensing/otherDatasets/sentinel_2/tif'
labels = os.listdir(images_path)
labels = sorted(labels)

labels

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

For faster processing, we will generate a copy of the dataset with only 10 images per class.

In [20]:
import shutil 

os.makedirs('data/EuroSAT-small/', exist_ok=True)
for label in labels:
    os.makedirs('data/EuroSAT-small/' + label, exist_ok=True)
    images = sorted(os.listdir(images_path + '/' + label))[:10]
    for image in images:
        shutil.copy(images_path + '/' + label + '/' + image, 'data/EuroSAT-small/' + label + '/' + image)

You can use the `STACGenerator` to create the STAC metadata for your dataset in the form of a dataframe. The item parser will depend on the structure of your dataset. We offer some predefined parsers for common datasets, but you can also create your own parser.

In [42]:
from eotdl.curation.stac.parsers import UnestructuredParser
from eotdl.curation.stac.stac import STACGenerator
from eotdl.curation.stac.dataframe_labeling import LabeledStrategy

stac_generator = STACGenerator(image_format='tif', item_parser=UnestructuredParser, labeling_strategy=LabeledStrategy)

df = stac_generator.get_stac_dataframe('data/EuroSAT-small')
df.head()


,image,label,ix,collection,extensions,bands
0,data/EuroSAT-small/River/River_1005.tif,River,0,data/EuroSAT-small/source,None,None
1,data/EuroSAT-small/River/River_1002.tif,River,0,data/EuroSAT-small/source,None,None
2,data/EuroSAT-small/River/River_1001.tif,River,0,data/EuroSAT-small/source,None,None
3,data/EuroSAT-small/River/River_1003.tif,River,0,data/EuroSAT-small/source,None,None
4,data/EuroSAT-small/River/River_1004.tif,River,0,data/EuroSAT-small/source,None,None


Now we save the STAC metadata. The `id` given to the STAC catalog will be used as the name of the dataset in EOTDL (which has the same requirements than can be found in the [documentation](/docs/datasets/ingest)).

In [39]:
output = 'data/EuroSAT-small-STAC'
stac_generator.generate_stac_metadata(
	stac_id='EuroSAT-Q1-small', 
	description='EuroSAT dataset (only 10 samples per class)', 
	stac_dataframe=df, 
	output_folder=output
)

Generating source collection...


100%|██████████| 27000/27000 [03:27<00:00, 129.84it/s]


Validating and saving catalog...
Success!


And, optionally, the labels using the labels extension.

In [40]:
from eotdl.curation.stac.extensions.label import ImageNameLabeler

catalog = output + '/catalog.json'
labels_extra_properties = {'label_properties': ["label"],
                          'label_methods': ["manual"],
                          'label_tasks': ["classification"]}

labeler = ImageNameLabeler()
labeler.generate_stac_labels(catalog, stac_dataframe=df, **labels_extra_properties)

Generating labels collection...


27000it [01:03, 426.66it/s]


Success on labels generation!


Once the STAC metadata is generated, we can ingest the dataset into EOTDL.

In [44]:
from eotdl.datasets import ingest_dataset

ingest_dataset('data/EuroSAT-small-STAC')

Loading STAC catalog...
New version created, version: 1


100%|██████████| 200/200 [00:51<00:00,  3.91it/s]


Ingesting STAC catalog...
Done


After the ingestion, you can explore and stage your dataset like shown in the previous tutorial.

In [8]:
from eotdl.datasets import retrieve_datasets

datasets = retrieve_datasets('EuroSAT')
datasets

['EuroSAT-Q1-small',
 'EuroSAT-RGB',
 'EuroSAT-RGB-Q1',
 'EuroSAT-RGB-Q2',
 'EuroSAT-RGB-STAC',
 'EuroSAT-STAC',
 'EuroSAT']

In [10]:
from eotdl.datasets import download_dataset

dst_path = download_dataset('EuroSAT-Q1-small', path='data')
dst_path

To download assets, set assets=True or -a in the CLI.


'data/EuroSAT-Q1-small/v1'

By default it will only download the STAC metadata. In case you also want to download the actual data, you can use the `assets` parameter. 

> The `force` parameter will overwrite the dataset if it already exists.

In [11]:
from eotdl.datasets import download_dataset

dst_path =  download_dataset('EuroSAT-Q1-small', path='data', assets=True, force=True)
dst_path

100%|██████████| 200/200 [00:49<00:00,  4.07it/s]


'data/EuroSAT-Q1-small/v1'

You will find the data in  the `assets` subfolder, where a subfolder for each items with its `id` will contain all the assets for that item.

In [12]:
from glob import glob

glob(dst_path + '/assets/*.tif')[:3]

['data/EuroSAT-Q1-small/v1/assets/Industrial_1000.tif',
 'data/EuroSAT-Q1-small/v1/assets/Industrial_10.tif',
 'data/EuroSAT-Q1-small/v1/assets/Highway_1.tif']

Alternatively, you can download an asset using its url.

In [16]:
import json

with open(dst_path + '/EuroSAT-Q1-small/source/AnnualCrop_1/AnnualCrop_1.json', 'r') as f:
	data = json.load(f)

data['assets']

{'AnnualCrop_1': {'href': 'https://api.eotdl.com/datasets/662a3c33038fa20bdd97ead2/download/AnnualCrop_1.tif',
  'type': 'image/tiff; application=geotiff',
  'title': 'AnnualCrop_1',
  'roles': ['data']}}

In [17]:
from eotdl.datasets import download_file_url

url = data['assets']['AnnualCrop_1']['href']
download_file_url(url, 'data')

'data/assets/AnnualCrop_1.tif'